In [ ]:
import torch

In [ ]:
torch.cuda.set_device(torch.device('cuda:0'))

In [ ]:
class NeuralNet(torch.nn.Module):
    
    def __init__(self, input_size, hidden_layer_size, output_size):
        super().__init__()
        
        self.layer_1 = torch.nn.Linear(input_size, hidden_layer_size)
        self.layer_2 = torch.nn.Linear(hidden_layer_size, output_size)
        self.relu = torch.nn.ReLU()
        self.activation = torch.nn.Sigmoid()
        
    def forward(self, input_x):
        hidden_res = self.layer_1(input_x)
        hidden_activ = self.relu(hidden_res)
        output = self.layer_2(hidden_activ)
        
        return self.activation(output)

In [ ]:
class Net(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(13, 50)
        self.relu1 = torch.nn.ReLU()
        self.out = torch.nn.Linear(50, 1)
        self.out_act = torch.nn.Sigmoid()
        
    def forward(self, input_):
        a1 = self.fc1(input_)
        h1 = self.relu1(a1)
        a2 = self.out(h1)
        y = self.out_act(a2)
        return y

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_DataLoaders_from_csv(file_name, y_column, batch_size=32, valid_size=0.25):
    df = pd.read_csv(file_name).astype(float)
    df = df.sample(frac=1)  # shuffle dataframe
    train_size, valid_size = int(df.shape[0]*(1-valid_size)), int(df.shape[0]*(valid_size))
    print(f"train size = {train_size}, validation size = {valid_size}")
    
    y_train_np_data, y_valid_np_data = df[y_column].head(train_size).values, df[y_column].tail(valid_size).values
    x_train_np_data, x_valid_np_data = (df.drop(columns=[y_column]).head(train_size).values, 
                                        df.drop(columns=[y_column]).head(valid_size).values)
    
    y_train_tensor, y_valid_tensor = torch.from_numpy(y_train_np_data).view(-1, 1), torch.from_numpy(y_valid_np_data).view(-1, 1)
    x_train_tensor, x_valid_tensor = torch.from_numpy(x_train_np_data), torch.from_numpy(x_valid_np_data)
    
    tensor_ds_train = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)  # create TRAIN tensor dataset
    tensor_ds_valid = torch.utils.data.TensorDataset(x_valid_tensor, y_valid_tensor)  # create VALID tensor dataset
    
    tensor_dl_train = torch.utils.data.DataLoader(tensor_ds_train, batch_size=batch_size, shuffle=True)
    tensor_dl_valid = torch.utils.data.DataLoader(tensor_ds_valid, batch_size=batch_size*2)
    
    return (tensor_dl_train, tensor_dl_valid)

In [ ]:
train_dl, valid_dl = get_DataLoaders_from_csv('train_transformed.csv', 'Survived')

In [ ]:
def preprocess(x, y, input_size, device):
    return x.view(-1, input_size), y


class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func
        self.dev =  torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    
    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for x_batch, y_batch in batches:
            yield (self.func(x_batch, y_batch, x_batch.shape[-1], self.dev))

In [ ]:
warped_train_dl = WrappedDataLoader(train_dl, preprocess)
warped_valid_dl = WrappedDataLoader(valid_dl, preprocess)